In [1]:
import numpy as np
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
import pandas as pd
import os
import re
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import time 
import threading
import psutil
import GPUtil
from time import sleep
from datetime import datetime

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def monitor_resources(interval, path_name):
    while not stop_thread.is_set():
        # Obter o timestamp atual
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Monitorando CPU e memória
        cpu_usage = psutil.cpu_percent(interval=interval)
        memory_info = psutil.virtual_memory()
        memory_usage = memory_info.used / (1024 ** 2)  # Convertendo de bytes para MB

        # Monitorando GPU
        gpus = GPUtil.getGPUs()
        gpu_info = [(gpu.load, gpu.memoryUsed) for gpu in gpus]
        gpu_load = gpu_info[0][0]
        gpu_memory_used = gpu_info[0][1]

        new_row = pd.DataFrame([{
            'Timestamp': current_time,
            'CPU': cpu_usage,
            'Memory': memory_usage,
            'GPU': gpu_load,
            'GPU_Memory': gpu_memory_used
        }])

        if not os.path.isfile(path_name):
            df = pd.DataFrame(columns=["Timestamp", "CPU", "Memory", "GPU", "GPU_Memory"])
            df.to_csv(path_name, index=False)
        else:
            df = pd.read_csv(path_name)

        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(path_name, index=False)

        # Limpando a memória
        del df

        sleep(interval)

In [ ]:
torch.cuda.set_per_process_memory_fraction(0.99)
config = Config(executors=[ThreadPoolExecutor()])
parsl.load(config)

In [ ]:
def extractName(text):
  return re.findall(r'"name"\s*:\s*"([^"]+)"',text)

def extractTag(text):
  matches = re.findall(r'\"tags\": \[(.*?)\]', text, re.DOTALL)
  matches = re.findall(r'\"(.*?)\"', matches[0].strip(','))
  return matches

def extractDescription(text):
  return re.findall(r'"description"\s*:\s*"([^"]+)"',text)

def extractLabel(text):
    return re.findall(r'"label"\s*:\s*"([^"]+)"', text)

def removeWords(text, words):
  for word in words:
    text = text.replace(word,"")
  return text

In [ ]:
os.chdir(os.path.join(os.pardir,os.pardir, 'workflows_galaxy'))
nomes_arquivos = []
diretorio = os.getcwd()
print(diretorio)
for item in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, item)
    if os.path.isfile(caminho_completo):
        nomes_arquivos.append(item)

print(len(nomes_arquivos))

In [ ]:
dic_workflows = {}
@python_app
def processar_arquivo(arquivo):
    # Carregar o arquivo JSON
    with open(arquivo, 'r') as file:
        fileName = file.name
        dados = file.read()
        matches_name = extractName(dados)
        matches_tag = extractTag(dados)
        matches_description = extractDescription(dados)
        matches_label = extractLabel(dados)

        combined_results = {
        'Tags': matches_tag if matches_tag else [],  
        'Descricao': ' '.join(matches_name + matches_tag + matches_description + matches_label),
        'Json': dados
        }
        dic_workflows[fileName] = combined_results

        return combined_results


futures = [processar_arquivo(arquivo) for arquivo in nomes_arquivos]
resultados = [future.result() for future in futures]

In [ ]:
# Função para converter texto em embedding
def text_to_embedding(text, tokenizer, device, model):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}  # Mover tensores para o dispositivo
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mover o resultado de volta para a CPU e converter para numpy

In [ ]:
os.chdir("..")
os.chdir("scr")
os.chdir("similaridade")

top_x_list = list(range(3,11))  
rounds = list(range(1,11))

execution_times = []

In [ ]:
for exec_round in rounds:
    df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()
    for top_x in top_x_list:
        # Medir o tempo de execução
        start_time = time.time()

        stop_thread = threading.Event()
        monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"hardware_info_bert/top{top_x}_descricao_round{exec_round}.csv",))    
        monitor_thread.daemon = True
        monitor_thread.start()

        # Carregar o modelo e o tokenizer
        tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
        model = BertModel.from_pretrained('google-bert/bert-large-uncased')

        # Definir o dispositivo (GPU ou CPU)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Supondo que você já tem um DataFrame df com uma coluna 'text'
        texts = df['Descricao'].tolist()

        # Converter todos os textos para embeddings
        embeddings = [text_to_embedding(text, tokenizer, device, model) for text in texts]

        # Converter a lista de embeddings em um array 2D
        embeddings_array = np.vstack(embeddings)

        # Calcular a matriz de similaridade
        similarity_matrix = cosine_similarity(embeddings_array)
        
        # Identificar os índices dos textos mais similares para cada texto
        similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

        # Remover o índice do próprio texto
        corrected_similar_indices = []
        corrected_similar_tags = []
        for idx, indices in enumerate(similar_indices):
            filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
            filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
            corrected_similar_indices.append(filtered_indices)
            corrected_similar_tags.append(filtered_tags)

        # Criar coluna no DataFrame para os índices dos textos mais similares
        df[f'top{top_x}_descricao'] = corrected_similar_indices

        # Criar coluna no DataFrame para as tags dos textos mais similares
        df[f'top{top_x}_tags_descricao'] = corrected_similar_tags

        # Calcular a média das similaridades dos textos mais similares para cada texto
        mean_similarities = []
        for idx, indices in enumerate(corrected_similar_indices):
            similarities = [similarity_matrix[idx, i] for i in indices]
            mean_similarity = np.mean(similarities)
            mean_similarities.append(mean_similarity)

        df[f'mean_similarity_top{top_x}_descricao'] = mean_similarities

        # Calcular o tempo de execução
        execution_time = time.time() - start_time

        # Adicionar o tempo de execução ao DataFrame de tempos de execução
        execution_times.append({'top_x': top_x, 'execution_time_descricao': execution_time})

        torch.cuda.empty_cache()
        stop_thread.set()
        monitor_thread.join()


        #####JSON####

        start_time = time.time()
        stop_thread = threading.Event()
        monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"hardware_info_bert/top{top_x}_json_round{exec_round}.csv",))
        monitor_thread.daemon = True
        monitor_thread.start()

        # Carregar o modelo e o tokenizer
        tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
        model = BertModel.from_pretrained('google-bert/bert-large-uncased')

        # Definir o dispositivo (GPU ou CPU)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Supondo que você já tem um DataFrame df com uma coluna 'text'
        texts = df['Json'].tolist()

        # Converter todos os textos para embeddings
        embeddings = [text_to_embedding(text, tokenizer, device, model) for text in texts]

        # Converter a lista de embeddings em um array 2D
        embeddings_array = np.vstack(embeddings)

        # Calcular a matriz de similaridade
        similarity_matrix = cosine_similarity(embeddings_array)
        
        # Identificar os índices dos textos mais similares para cada texto
        similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

        # Remover o índice do próprio texto
        corrected_similar_indices = []
        corrected_similar_tags = []
        for idx, indices in enumerate(similar_indices):
            filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
            filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
            corrected_similar_indices.append(filtered_indices)
            corrected_similar_tags.append(filtered_tags)

        # Criar coluna no DataFrame para os índices dos textos mais similares
        df[f'top{top_x}_json'] = corrected_similar_indices

        # Criar coluna no DataFrame para as tags dos textos mais similares
        df[f'top{top_x}_tags_json'] = corrected_similar_tags

        # Calcular a média das similaridades dos textos mais similares para cada texto
        mean_similarities = []
        for idx, indices in enumerate(corrected_similar_indices):
            similarities = [similarity_matrix[idx, i] for i in indices]
            mean_similarity = np.mean(similarities)
            mean_similarities.append(mean_similarity)

        df[f'mean_similarity_top{top_x}_json'] = mean_similarities

        # Calcular o tempo de execução
        execution_time = time.time() - start_time

        # Adicionar o tempo de execução ao DataFrame de tempos de execução
        execution_times.append({'top_x': top_x, 'execution_time_json': execution_time})

        torch.cuda.empty_cache()
        stop_thread.set()
        monitor_thread.join()

    df.to_csv(f"infos_bert/results_bert_round{exec_round}.csv")

In [ ]:
def text_to_embedding(text, tokenizer, device, model):
    inputs  = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    return embedding

In [ ]:
for exec_round in rounds:
    df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()
    for top_x in top_x_list:
        # Medir o tempo de execução
        start_time = time.time()

        stop_thread = threading.Event()
        monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"hardware_info_scibert/top{top_x}_descricao_round{exec_round}.csv",))    
        monitor_thread.daemon = True
        monitor_thread.start()

        # Carregar o modelo e o tokenizer
        tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

        # Definir o dispositivo (GPU ou CPU)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Supondo que você já tem um DataFrame df com uma coluna 'text'
        texts = df['Descricao'].tolist()

        # Converter todos os textos para embeddings
        embeddings = [text_to_embedding(text, tokenizer, device, model) for text in texts]

        # Converter a lista de embeddings em um array 2D
        embeddings_array = np.vstack(embeddings)

        # Calcular a matriz de similaridade
        similarity_matrix = cosine_similarity(embeddings_array)
        
        # Identificar os índices dos textos mais similares para cada texto
        similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

        # Remover o índice do próprio texto
        corrected_similar_indices = []
        corrected_similar_tags = []
        for idx, indices in enumerate(similar_indices):
            filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
            filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
            corrected_similar_indices.append(filtered_indices)
            corrected_similar_tags.append(filtered_tags)

        # Criar coluna no DataFrame para os índices dos textos mais similares
        df[f'top{top_x}_descricao'] = corrected_similar_indices

        # Criar coluna no DataFrame para as tags dos textos mais similares
        df[f'top{top_x}_tags_descricao'] = corrected_similar_tags

        # Calcular a média das similaridades dos textos mais similares para cada texto
        mean_similarities = []
        for idx, indices in enumerate(corrected_similar_indices):
            similarities = [similarity_matrix[idx, i] for i in indices]
            mean_similarity = np.mean(similarities)
            mean_similarities.append(mean_similarity)

        df[f'mean_similarity_top{top_x}_descricao'] = mean_similarities

        # Calcular o tempo de execução
        execution_time = time.time() - start_time

        # Adicionar o tempo de execução ao DataFrame de tempos de execução
        execution_times.append({'top_x': top_x, 'execution_time_descricao': execution_time})

        torch.cuda.empty_cache()
        stop_thread.set()
        monitor_thread.join()


        #####JSON####

        start_time = time.time()
        stop_thread = threading.Event()
        monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"hardware_info_scibert/top{top_x}_json_round{exec_round}.csv",))
        monitor_thread.daemon = True
        monitor_thread.start()

        # Carregar o modelo e o tokenizer
        tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

        # Definir o dispositivo (GPU ou CPU)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Supondo que você já tem um DataFrame df com uma coluna 'text'
        texts = df['Json'].tolist()

        # Converter todos os textos para embeddings
        embeddings = [text_to_embedding(text, tokenizer, device, model) for text in texts]

        # Converter a lista de embeddings em um array 2D
        embeddings_array = np.vstack(embeddings)

        # Calcular a matriz de similaridade
        similarity_matrix = cosine_similarity(embeddings_array)
        
        # Identificar os índices dos textos mais similares para cada texto
        similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

        # Remover o índice do próprio texto
        corrected_similar_indices = []
        corrected_similar_tags = []
        for idx, indices in enumerate(similar_indices):
            filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
            filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
            corrected_similar_indices.append(filtered_indices)
            corrected_similar_tags.append(filtered_tags)

        # Criar coluna no DataFrame para os índices dos textos mais similares
        df[f'top{top_x}_json'] = corrected_similar_indices

        # Criar coluna no DataFrame para as tags dos textos mais similares
        df[f'top{top_x}_tags_json'] = corrected_similar_tags

        # Calcular a média das similaridades dos textos mais similares para cada texto
        mean_similarities = []
        for idx, indices in enumerate(corrected_similar_indices):
            similarities = [similarity_matrix[idx, i] for i in indices]
            mean_similarity = np.mean(similarities)
            mean_similarities.append(mean_similarity)

        df[f'mean_similarity_top{top_x}_json'] = mean_similarities

        # Calcular o tempo de execução
        execution_time = time.time() - start_time

        # Adicionar o tempo de execução ao DataFrame de tempos de execução
        execution_times.append({'top_x': top_x, 'execution_time_json': execution_time})

        torch.cuda.empty_cache()
        stop_thread.set()
        monitor_thread.join()

    df.to_csv(f"infos_scibert/results_scibert_round{exec_round}.csv")